## scDEF on 3k PBMCs
In this notebook we apply scDEF to the 3k PBMCs data set from 10X Genomics, which is available through Scanpy.

## Imports and setup

In [ ]:
import scdef

import numpy as np
import pandas as pd
import scanpy as sc

sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()

## Load and pre-process data
Just like in the Scanpy tutorial. Note that we keep the raw counts in `adata.layers['counts']`.

In [ ]:
# !mkdir data
# !wget http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz -O data/pbmc3k_filtered_gene_bc_matrices.tar.gz
# !cd data; tar -xzf pbmc3k_filtered_gene_bc_matrices.tar.gz
# !mkdir write

In [ ]:
adata = sc.read_10x_mtx(
    'data/filtered_gene_bc_matrices/hg19/',  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)                              # write a cache file for faster subsequent reading
adata.obs.index = adata.obs.index.str.replace('-1', '')
adata.var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`

sc.pp.filter_cells(adata, min_genes=2)
sc.pp.filter_genes(adata, min_cells=3)
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
adata = adata[adata.obs.n_genes_by_counts < 2500, :]
adata = adata[adata.obs.pct_counts_mt < 5, :]
adata.raw = adata

adata.layers['counts'] = adata.X.toarray() # Keep the counts, for scDEf

# Keep only HVGs
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5) 

adata = adata[:, adata.var.highly_variable]

# Process and visualize the data
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
sc.tl.umap(adata)
sc.tl.leiden(adata)
sc.pl.umap(adata, color=['leiden'])

## Run scDEF
We first create ann scDEF object with default parameters. The number of layers, their sizes and other model parameters can be easily inspected by printing the object. The default scDEF contains 3 layers of sizes 60, 30, 15 (from lower to upper levels). After fitting the model, the ARD posteriors can be used to filter out irrelevant factors.

In [ ]:
scd = scdef.scDEF(adata, counts_layer='counts') # if layer='counts' is not used, you must ensure adata.X contains the counts
print(scd) # inspect the scDEF object, which contains a copy of the input AnnData

In [ ]:
scd.learn(n_epoch=5) # learn the hierarchical gene signatures

As the logs indicate, the `AnnData` object inside the `scDEF` object has been updated with annotations that can be used as cluster assignments at different levels of resolution. By default, scDEF uses three layers, so we have three such annotations: `factor` for layer 0 (the lower level), `hfactor` for layer 1, and `hhfactor` for layer 2 (the upper level). Let's look at the UMAP coloured by these cluster assignments.

In [ ]:
sc.pl.umap(scd.adata, color=['factor', 'hfactor', 'hhfactor'], frameon=False)

By default, the factors at each layer with the 50% top posterior relevancies are kept. However, we may want to keep more or less than this. We can check the relevancies with the `plot_ard` method and adjust our filtering with the `filter` method, which will update all the annotations accordingly. We can either use a single threshold for all layers, or use a layer-specific threshold.

In [ ]:
scd.plot_ard(ard=[.7, .8, .8], figsize=(16,4))
scd.filter_factors([.7, .8, .8])

Let's look at the clustering assignments again with this new filter.

In [ ]:
sc.pl.umap(scd.adata, color=['factor', 'hfactor', 'hhfactor'], frameon=False)

We can also look at the graph containing the factor hierarchy with `scDEF.make_graph`, which allows for flexible customization. 

In [ ]:
scd.make_graph(filled='factor', show_signatures=False) # simple graph with color-filled nodes
scd.graph # Graphviz object

In [ ]:
scd.make_graph(show_signatures=True) # simple graph with gene signatures in nodes
scd.graph

It is also possible to get the gene signatures of different levels as lists with the `scDEF.get_rankings` method. The output of this is a list of genes for each factor in layer `layer_idx`, sorted by decreasing weight in the factor.

In [ ]:
gene_programs = scd.get_rankings(layer_idx=0)

We can use `dotplot` to visualize the asssociation of the scDEF factors at different layers with annotations from the `AnnData` object.

In [ ]:
# Compare with leiden clusters
scd.plot_obs_factor_dotplot('leiden', 0, titlesize=16, figsize=(16,3)) # layer 0
scd.plot_obs_factor_dotplot('leiden', 1, titlesize=16, figsize=(16,3)) # layer 1
scd.plot_obs_factor_dotplot('leiden', 2, titlesize=16, figsize=(16,3)) # layer 2

Finally, we can use the hierarchical structure provided by the scDEF to visualize the data in a PAGA graph (Wolf et al, 2018). This makes the different levels of resolution and their relationships evident. We set `reuse_pos=True` to make initialize the positions of the nodes in the graph at layer `i` with the positions of the nodes from layer `i+1`.

In [ ]:
scd.plot_multilevel_paga(figsize=(16,4), reuse_pos=True, frameon=False)